In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import xgboost as xgb
import optuna
import optuna.visualization as vis

n = 9

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):

    if trial.should_prune():
        raise optuna.TrialPruned()

    ### ========== Shuffle and Split Data ========== ###

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.8, random_state=42)  # 60% training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.8, random_state=42)  # 20% validation, 20% test

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    use_pca = trial.suggest_categorical('use_pca', ['True', 'False'])
    n_components_pca = trial.suggest_int('n_pca_components', 1, max(1, X_train_scaled.shape[1]))


    ### ========== Train and Evaluate Model ========== ###

    if use_pca == 'True':
        pca = PCA(
            n_components=n_components_pca
        )
        X_train_scaled = pca.fit_transform(X_train_scaled)
        X_val_scaled = pca.transform(X_val)
        X_test_scaled = pca.transform(X_test)

    params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=25),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'n_jobs': -1
        }

    # Initialize and train classifier
    classifier = xgb.XGBClassifier(**params)

    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    # print(f"Validation Accuracy: {val_accuracy:.4f}")

    # Evaluate on the test set
    y_test_pred = classifier.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return val_accuracy


# Create a study object
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()
# vis.plot_intermediate_values(study)

[I 2024-11-11 19:19:10,743] A new study created in memory with name: no-name-73712545-e992-474c-872e-246f869e9d76
[I 2024-11-11 19:19:11,008] Trial 0 finished with value: 0.5010416666666667 and parameters: {'use_pca': 'False', 'n_pca_components': 2, 'n_estimators': 850, 'max_depth': 3, 'learning_rate': 0.16381553414696418, 'subsample': 0.500745775238304, 'colsample_bytree': 0.9757307338993106, 'gamma': 1.8202260775291301, 'min_child_weight': 9}. Best is trial 0 with value: 0.5010416666666667.
[I 2024-11-11 19:19:11,077] Trial 1 finished with value: 0.4895833333333333 and parameters: {'use_pca': 'False', 'n_pca_components': 8, 'n_estimators': 275, 'max_depth': 10, 'learning_rate': 0.12828841467704616, 'subsample': 0.990500097833556, 'colsample_bytree': 0.7022195429832396, 'gamma': 4.069975823839852, 'min_child_weight': 6}. Best is trial 0 with value: 0.5010416666666667.


Test Accuracy: 0.5010
Test Accuracy: 0.5054


[I 2024-11-11 19:19:11,269] Trial 2 finished with value: 0.4986111111111111 and parameters: {'use_pca': 'False', 'n_pca_components': 3, 'n_estimators': 225, 'max_depth': 7, 'learning_rate': 0.012873581281138557, 'subsample': 0.5418287527181729, 'colsample_bytree': 0.7783085253473125, 'gamma': 1.54886598295775, 'min_child_weight': 3}. Best is trial 0 with value: 0.5010416666666667.


Test Accuracy: 0.5000
Test Accuracy: 0.4990


[I 2024-11-11 19:19:11,454] Trial 3 finished with value: 0.48993055555555554 and parameters: {'use_pca': 'False', 'n_pca_components': 1, 'n_estimators': 950, 'max_depth': 7, 'learning_rate': 0.22185929263550513, 'subsample': 0.9126962312877088, 'colsample_bytree': 0.7681313905260758, 'gamma': 4.980562413973632, 'min_child_weight': 8}. Best is trial 0 with value: 0.5010416666666667.
[I 2024-11-11 19:19:11,613] Trial 4 finished with value: 0.484375 and parameters: {'use_pca': 'True', 'n_pca_components': 2, 'n_estimators': 925, 'max_depth': 6, 'learning_rate': 0.1808865028418688, 'subsample': 0.9804992248182063, 'colsample_bytree': 0.6209453491577708, 'gamma': 0.7952936473297612, 'min_child_weight': 2}. Best is trial 0 with value: 0.5010416666666667.
[I 2024-11-11 19:19:11,742] Trial 5 finished with value: 0.5079861111111111 and parameters: {'use_pca': 'False', 'n_pca_components': 6, 'n_estimators': 475, 'max_depth': 6, 'learning_rate': 0.11023538155276708, 'subsample': 0.5821751675509483

...

Test Accuracy: 0.4997
Test Accuracy: 0.5001


[I 2024-11-11 19:19:26,063] Trial 96 finished with value: 0.5204861111111111 and parameters: {'use_pca': 'False', 'n_pca_components': 6, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.08066565106575409, 'subsample': 0.5285664703678903, 'colsample_bytree': 0.684467168981229, 'gamma': 2.466423210059281, 'min_child_weight': 8}. Best is trial 64 with value: 0.5239583333333333.
[I 2024-11-11 19:19:26,183] Trial 97 finished with value: 0.5125 and parameters: {'use_pca': 'False', 'n_pca_components': 6, 'n_estimators': 225, 'max_depth': 3, 'learning_rate': 0.08169382080861645, 'subsample': 0.528099320736802, 'colsample_bytree': 0.8674901330248097, 'gamma': 2.1664304456171606, 'min_child_weight': 8}. Best is trial 64 with value: 0.5239583333333333.


Test Accuracy: 0.5016
Test Accuracy: 0.5019


[I 2024-11-11 19:19:26,300] Trial 98 finished with value: 0.5076388888888889 and parameters: {'use_pca': 'False', 'n_pca_components': 6, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.06872773594696467, 'subsample': 0.5099043370188685, 'colsample_bytree': 0.6822859062410329, 'gamma': 1.335199098780267, 'min_child_weight': 7}. Best is trial 64 with value: 0.5239583333333333.
[I 2024-11-11 19:19:26,426] Trial 99 finished with value: 0.5006944444444444 and parameters: {'use_pca': 'False', 'n_pca_components': 7, 'n_estimators': 250, 'max_depth': 3, 'learning_rate': 0.06039040188875827, 'subsample': 0.5415200517314215, 'colsample_bytree': 0.6667718294744542, 'gamma': 2.389690340580567, 'min_child_weight': 8}. Best is trial 64 with value: 0.5239583333333333.


Test Accuracy: 0.4966
Test Accuracy: 0.4960
Best hyperparameters: {'use_pca': 'False', 'n_pca_components': 6, 'n_estimators': 475, 'max_depth': 5, 'learning_rate': 0.06573112349097421, 'subsample': 0.7092188509117748, 'colsample_bytree': 0.7749961483199131, 'gamma': 3.74455549146501, 'min_child_weight': 3}
Best value: 0.5239583333333333
